In [ ]:
!pip install cryptography

In [1]:
import datetime
from cryptography import x509
from cryptography.x509.oid import NameOID
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.serialization import pkcs12

# --- SETUP: GENERATE DUMMY DATA ---
# 1. Generate a Private Key
private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)

# 2. Create a Self-Signed Certificate
subject = issuer = x509.Name([
    x509.NameAttribute(NameOID.COMMON_NAME, u"Demo User"),
])
cert = x509.CertificateBuilder().subject_name(
    subject
).issuer_name(
    issuer
).public_key(
    private_key.public_key()
).serial_number(
    x509.random_serial_number()
).not_valid_before(
    datetime.datetime.now(datetime.timezone.utc)
).not_valid_after(
    datetime.datetime.now(datetime.timezone.utc) + datetime.timedelta(days=365)
).sign(private_key, hashes.SHA256())

# --- PKCS#12 DEMO ---
password = b"mypassword"
p12_filename = "pkcs12_test.p12"

print(f"--- Creating {p12_filename} ---")
# 3. Create (Serialize) the PKCS#12 container
encryption = serialization.BestAvailableEncryption(password)
p12_data = pkcs12.serialize_key_and_certificates(
    name=b"Demo Identity",
    key=private_key,
    cert=cert,
    cas=None,
    encryption_algorithm=encryption
)

with open(p12_filename, "wb") as f:
    f.write(p12_data)

print(f"Successfully saved {p12_filename}")

print("\n--- Reading data back from the .p12 file ---")
# 4. Read (Load) the PKCS#12 container
with open(p12_filename, "rb") as f:
    data = f.read()
    
# Returns a tuple: (private_key, certificate, additional_cas)
loaded_key, loaded_cert, loaded_cas = pkcs12.load_key_and_certificates(
    data, 
    password
)

print(f"Loaded Certificate Subject: {loaded_cert.subject.get_attributes_for_oid(NameOID.COMMON_NAME)[0].value}")
print(f"Loaded Key Type: {type(loaded_key).__name__}")

--- Creating pkcs12_test.p12 ---
Successfully saved pkcs12_test.p12

--- Reading data back from the .p12 file ---
Loaded Certificate Subject: Demo User
Loaded Key Type: RSAPrivateKey
